### Part 1. Read in and Join CitiBike CSV Files

In [1]:
# Lib
import pandas as pd 
import numpy as np
import os 
import requests
import json
from datetime import datetime

In [2]:
# Create a list of full file paths for all CSVs in the Data/ folder

folderpath = "Data"
filepaths = [os.path.join(folderpath, name) for name in os.listdir(folderpath)]

In [3]:
# Read and merge all CSV files into a single DataFrame using a generator and pd.concat()

df = pd.concat((pd.read_csv(f) for f in filepaths), ignore_index=True)

In [4]:
# This line reads each CSV file from the filepaths list using pd.read_csv() inside a generator.
# The generator feeds the resulting DataFrames into pd.concat(), which combines them vertically into one single DataFrame.
# ignore_index=True resets the row index in the final combined DataFrame.

In [5]:
# checking 

df.shape

(895485, 13)

In [6]:
df.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,3255D3E3F33CDC45,classic_bike,2022-03-18 15:38:17,2022-03-18 15:45:34,Mama Johnson Field - 4 St & Jackson St,HB404,South Waterfront Walkway - Sinatra Dr & 1 St,HB103,40.743140,-74.040041,40.736982,-74.027781,casual
1,17FA5604A37338F9,electric_bike,2022-03-04 16:44:48,2022-03-04 16:50:45,Baldwin at Montgomery,JC020,Grove St PATH,JC005,40.723659,-74.064194,40.719586,-74.043117,member
2,7DEC9ADDB8D6BBE1,electric_bike,2022-03-13 17:44:32,2022-03-13 17:54:44,Baldwin at Montgomery,JC020,Grove St PATH,JC005,40.723659,-74.064194,40.719586,-74.043117,member
3,9D69F74EEF231A2E,classic_bike,2022-03-13 15:33:47,2022-03-13 15:41:22,Baldwin at Montgomery,JC020,Grove St PATH,JC005,40.723659,-74.064194,40.719586,-74.043117,member
4,C84AE4A9D78A6347,classic_bike,2022-03-11 12:21:18,2022-03-11 12:33:24,Baldwin at Montgomery,JC020,Grove St PATH,JC005,40.723659,-74.064194,40.719586,-74.043117,member


In [7]:
# consistensy check

In [8]:
df.columns[df.columns.duplicated()]

Index([], dtype='object')

In [9]:
df.isnull().sum().sort_values(ascending=False)

end_station_name      3204
end_station_id        3204
end_lat               1970
end_lng               1970
start_station_name      10
start_station_id        10
ride_id                  0
rideable_type            0
started_at               0
ended_at                 0
start_lat                0
start_lng                0
member_casual            0
dtype: int64

### Step 2. Scap data using API

In [10]:
# NOAA API token
Token = "swmFOCMQXwkQSYVvGShcOcvdItUnAjxT"

In [14]:
url = "https://www.ncdc.noaa.gov/cdo-web/api/v2/data"
params = {
    "datasetid": "GHCND",
    "datatypeid": "TAVG",
    "stationid": "GHCND:USW00094789",  # LaGuardia Airport (NYC)
    "startdate": "2022-01-01",
    "enddate": "2022-12-31",
    "limit": 1000,
    "units": "metric"
}
headers = {
    "token": Token
}

r = requests.get(url, params=params, headers=headers)
data = json.loads(r.text)
data["results"][:3]

[{'date': '2022-01-01T00:00:00',
  'datatype': 'TAVG',
  'station': 'GHCND:USW00094789',
  'attributes': 'H,,S,',
  'value': 10.1},
 {'date': '2022-01-02T00:00:00',
  'datatype': 'TAVG',
  'station': 'GHCND:USW00094789',
  'attributes': 'H,,S,',
  'value': 11.5},
 {'date': '2022-01-03T00:00:00',
  'datatype': 'TAVG',
  'station': 'GHCND:USW00094789',
  'attributes': 'H,,S,',
  'value': 1.2}]

In [18]:
data = json.loads(r.text)

In [15]:
# Each dictionary in this list represents one day of weather data from LaGuardia Airport in 2022.
# 
# - 'date': The observation date in ISO format.
# - 'datatype': Always 'TAVG' in this filtered list, representing the average daily temperature.
# - 'station': NOAA station ID (GHCND:USW00094789 = LaGuardia Airport, NYC).
# - 'attributes': Data quality flags (can be ignored).
# - 'value': The actual average temperature for the day, in degrees Celsius.
#
# Example: On '2022-01-01', the average temperature at LaGuardia was 10.1°C.

In [23]:
# Keep only entries with average temperature (TAVG)
avg_temps = [item for item in data["results"] if item["datatype"] == "TAVG"]

In [24]:
# Build the DataFrame directly from the filtered list
df_temp = pd.DataFrame({
    "date": [datetime.strptime(item["date"], "%Y-%m-%dT%H:%M:%S") for item in avg_temps],
    "avg_temp": [float(item["value"]) for item in avg_temps]  # already in °C, no need to divide
})

In [25]:
# Save the DataFrame to CSV for reuse
df_temp.to_csv("Data/laguardia_weather_2022.csv", index=False)

### Step 3. Merging data

In [32]:
df["started_at"] = pd.to_datetime(df["started_at"]) # Convert the 'started_at' column from string to datetime format to enable time-based operations and merging

In [33]:
df["started_at"].dtype # check

dtype('<M8[ns]')

In [30]:
df["date"] = df["started_at"].dt.date #extracting just date part

In [34]:
# Merge CitiBike ride data with daily weather data using the 'date' column

In [37]:
df.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,date
0,3255D3E3F33CDC45,classic_bike,2022-03-18 15:38:17,2022-03-18 15:45:34,Mama Johnson Field - 4 St & Jackson St,HB404,South Waterfront Walkway - Sinatra Dr & 1 St,HB103,40.743140,-74.040041,40.736982,-74.027781,casual,2022-03-18
1,17FA5604A37338F9,electric_bike,2022-03-04 16:44:48,2022-03-04 16:50:45,Baldwin at Montgomery,JC020,Grove St PATH,JC005,40.723659,-74.064194,40.719586,-74.043117,member,2022-03-04
2,7DEC9ADDB8D6BBE1,electric_bike,2022-03-13 17:44:32,2022-03-13 17:54:44,Baldwin at Montgomery,JC020,Grove St PATH,JC005,40.723659,-74.064194,40.719586,-74.043117,member,2022-03-13
3,9D69F74EEF231A2E,classic_bike,2022-03-13 15:33:47,2022-03-13 15:41:22,Baldwin at Montgomery,JC020,Grove St PATH,JC005,40.723659,-74.064194,40.719586,-74.043117,member,2022-03-13
4,C84AE4A9D78A6347,classic_bike,2022-03-11 12:21:18,2022-03-11 12:33:24,Baldwin at Montgomery,JC020,Grove St PATH,JC005,40.723659,-74.064194,40.719586,-74.043117,member,2022-03-11


In [39]:
# Convert both to full datetime format
df["date"] = pd.to_datetime(df["date"])
df_temp["date"] = pd.to_datetime(df_temp["date"])

In [40]:
df_merged = df.merge(df_temp, how="left", on="date")

In [41]:
df_merged

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,date,avg_temp
0,3255D3E3F33CDC45,classic_bike,2022-03-18 15:38:17,2022-03-18 15:45:34,Mama Johnson Field - 4 St & Jackson St,HB404,South Waterfront Walkway - Sinatra Dr & 1 St,HB103,40.743140,-74.040041,40.736982,-74.027781,casual,2022-03-18,12.2
1,17FA5604A37338F9,electric_bike,2022-03-04 16:44:48,2022-03-04 16:50:45,Baldwin at Montgomery,JC020,Grove St PATH,JC005,40.723659,-74.064194,40.719586,-74.043117,member,2022-03-04,-2.7
2,7DEC9ADDB8D6BBE1,electric_bike,2022-03-13 17:44:32,2022-03-13 17:54:44,Baldwin at Montgomery,JC020,Grove St PATH,JC005,40.723659,-74.064194,40.719586,-74.043117,member,2022-03-13,-2.3
3,9D69F74EEF231A2E,classic_bike,2022-03-13 15:33:47,2022-03-13 15:41:22,Baldwin at Montgomery,JC020,Grove St PATH,JC005,40.723659,-74.064194,40.719586,-74.043117,member,2022-03-13,-2.3
4,C84AE4A9D78A6347,classic_bike,2022-03-11 12:21:18,2022-03-11 12:33:24,Baldwin at Montgomery,JC020,Grove St PATH,JC005,40.723659,-74.064194,40.719586,-74.043117,member,2022-03-11,4.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
895480,D438F1622839AC50,classic_bike,2022-12-06 15:43:38,2022-12-06 15:53:57,Dey St,JC065,Riverview Park,JC057,40.737828,-74.067083,40.744319,-74.043991,member,2022-12-06,8.3
895481,747A63A8E782D171,electric_bike,2022-12-08 08:17:51,2022-12-08 08:23:33,9 St HBLR - Jackson St & 8 St,HB305,City Hall - Washington St & 1 St,HB105,40.747907,-74.038412,40.737360,-74.030970,casual,2022-12-08,11.4
895482,AE090858CFDE6E82,electric_bike,2022-12-23 14:10:07,2022-12-23 14:14:18,Mama Johnson Field - 4 St & Jackson St,HB404,City Hall - Washington St & 1 St,HB105,40.743140,-74.040041,40.737360,-74.030970,member,2022-12-23,7.4
895483,B3CC8E70AF4E259C,classic_bike,2022-12-02 04:43:25,2022-12-02 04:46:55,Mama Johnson Field - 4 St & Jackson St,HB404,City Hall - Washington St & 1 St,HB105,40.743135,-74.040080,40.737360,-74.030970,member,2022-12-02,2.9


In [43]:
df_merged["avg_temp"].isnull().sum() # amazing match

0